In [ ]:
## Import required libraries and Objects

In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas
from datetime import date, datetime, timedelta

In [ ]:
## Using the Apewisdom API, we are pulling the top 100 stocks in order of most 'mentions' on Reddit. 

In [2]:
top_100_mentions_stocks = requests.get('https://apewisdom.io/api/v1.0/filter/wallstreetbets').json()['results'][:100]

In [ ]:
## Usinig above data, we are selecting for top 5 results. 

In [3]:
top_100_mentions_stocks[0:5]

[{'rank': 1,
  'ticker': 'SPY',
  'name': 'SPDR S&amp;P 500 ETF Trust',
  'mentions': '129',
  'upvotes': '629',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '307'},
 {'rank': 2,
  'ticker': 'TSLA',
  'name': 'Tesla',
  'mentions': '98',
  'upvotes': '819',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '339'},
 {'rank': 3,
  'ticker': 'TA',
  'name': 'TravelCenters of America',
  'mentions': '62',
  'upvotes': '10404',
  'rank_24h_ago': '26',
  'mentions_24h_ago': '14'},
 {'rank': 4,
  'ticker': 'GME',
  'name': 'GameStop',
  'mentions': '57',
  'upvotes': '300',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '97'},
 {'rank': 5,
  'ticker': 'NVDA',
  'name': 'NVIDIA',
  'mentions': '38',
  'upvotes': '319',
  'rank_24h_ago': '6',
  'mentions_24h_ago': '60'}]

In [ ]:
## Create a Pandas dataframe and drop all NA data using pandas function dropna.(organize and clean data)

In [4]:
top_100_mentions_stocks_df = pd.DataFrame.from_records(top_100_mentions_stocks)
top_100_mentions_stocks_df.dropna()
top_100_mentions_stocks_df

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,129,629,2,307
1,2,TSLA,Tesla,98,819,1,339
2,3,TA,TravelCenters of America,62,10404,26,14
3,4,GME,GameStop,57,300,3,97
4,5,NVDA,NVIDIA,38,319,6,60
...,...,...,...,...,...,...,...
95,96,DOW,Dow,2,7,48,9
96,97,KO,Coca-Cola,2,5,87,6
97,98,OG,Onion Global,2,4,77,5
98,99,RIVN,Rivian,2,17,38,5


In [ ]:
## Redefine the 'Mentions' column as a 'float' type of data. Select the 'mentions' data from the previous 24 hours to use as our desired data for evaluation.

In [5]:
top_100_mentions_stocks_df['mentions'] = top_100_mentions_stocks_df['mentions'].astype('float')
top_100_mentions_stocks_df['mentions_24h_ago'] = top_100_mentions_stocks_df['mentions_24h_ago'].astype('float')

In [6]:
top_100_mentions_stocks_df

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,129.0,629,2,307.0
1,2,TSLA,Tesla,98.0,819,1,339.0
2,3,TA,TravelCenters of America,62.0,10404,26,14.0
3,4,GME,GameStop,57.0,300,3,97.0
4,5,NVDA,NVIDIA,38.0,319,6,60.0
...,...,...,...,...,...,...,...
95,96,DOW,Dow,2.0,7,48,9.0
96,97,KO,Coca-Cola,2.0,5,87,6.0
97,98,OG,Onion Global,2.0,4,77,5.0
98,99,RIVN,Rivian,2.0,17,38,5.0


In [ ]:
## filter out unwanted data

In [7]:
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['mentions_24h_ago'] > 2]
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['ticker'] != 'SPY']
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['ticker'] != 'QQQ']
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['ticker'] != 'SQQQ']
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['ticker'] != 'TQQQ']
top_100_mentions_stocks_df = top_100_mentions_stocks_df[top_100_mentions_stocks_df['ticker'] != 'VIX']

In [ ]:
## Create variable mentions_delta_score to find the absolute value by using python function abs. For floating numbers - floating absolute value is returned

In [8]:
mentions_delta_score = abs(top_100_mentions_stocks_df['mentions'] - top_100_mentions_stocks_df['mentions_24h_ago']) / top_100_mentions_stocks_df['mentions_24h_ago'] 

In [ ]:
## Insert mentions volatitlity as a measurement to compare later, duplicates allowed.

In [9]:
if 'mentions_volatility' not in top_100_mentions_stocks_df.columns:
    top_100_mentions_stocks_df.insert(3, 'mentions_volatility', mentions_delta_score, True)
top_100_mentions_stocks_df

,rank,ticker,name,mentions_volatility,mentions,upvotes,rank_24h_ago,mentions_24h_ago
1,2,TSLA,Tesla,0.710914,98.0,819,1,339.0
2,3,TA,TravelCenters of America,3.428571,62.0,10404,26,14.0
3,4,GME,GameStop,0.412371,57.0,300,3,97.0
4,5,NVDA,NVIDIA,0.366667,38.0,319,6,60.0
5,6,AAPL,Apple,0.775862,26.0,138,4,116.0
...,...,...,...,...,...,...,...,...
95,96,DOW,Dow,0.777778,2.0,7,48,9.0
96,97,KO,Coca-Cola,0.666667,2.0,5,87,6.0
97,98,OG,Onion Global,0.600000,2.0,4,77,5.0
98,99,RIVN,Rivian,0.600000,2.0,17,38,5.0


In [ ]:
## Sort list based on highest volatility 

In [10]:
top_100_mentions_stocks_df.sort_values('mentions_volatility', ascending=False, inplace=True)
top_100_mentions_stocks_df.head(10)

,rank,ticker,name,mentions_volatility,mentions,upvotes,rank_24h_ago,mentions_24h_ago
2,3,TA,TravelCenters of America,3.428571,62.0,10404,26,14.0
20,21,AC,Associated Capital Group,2.333333,10.0,36,759,3.0
22,23,DNA,Ginkgo Bioworks,2.333333,10.0,403,175,3.0
25,26,LULU,lululemon athletica,2.000000,9.0,35,581,3.0
24,25,RNA,Avidity Biosciences,1.250000,9.0,-60,106,4.0
94,95,DE,Deere & Company,0.933333,2.0,13,8,30.0
80,81,AI,C3 AI,0.842105,3.0,20,27,19.0
82,83,GM,General Motors,0.785714,3.0,12,40,14.0
95,96,DOW,Dow,0.777778,2.0,7,48,9.0
5,6,AAPL,Apple,0.775862,26.0,138,4,116.0


In [ ]:
## Replace index with ticker

In [11]:
top_100_mentions_stocks_df.set_index('ticker', inplace=True)

In [12]:
top_100_mentions_stocks_df

,rank,name,mentions_volatility,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,,
TA,3,TravelCenters of America,3.428571,62.0,10404,26,14.0
AC,21,Associated Capital Group,2.333333,10.0,36,759,3.0
DNA,23,Ginkgo Bioworks,2.333333,10.0,403,175,3.0
LULU,26,lululemon athletica,2.000000,9.0,35,581,3.0
RNA,25,Avidity Biosciences,1.250000,9.0,-60,106,4.0
...,...,...,...,...,...,...,...
UP,17,Wheels Up,0.071429,13.0,70,46,14.0
DM,79,Desktop Metal,0.000000,3.0,7,79,3.0
MA,62,Mastercard,0.000000,4.0,100,133,4.0


In [ ]:
## Create dataframe from dictionary.(Creates DataFrame object from dictionary by columns or by index allowing dtype specification)

In [13]:
top_100_mentions_stocks_df = pd.DataFrame.from_dict(top_100_mentions_stocks_df, orient='columns')
top_100_mentions_stocks_df

,rank,name,mentions_volatility,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,,
TA,3,TravelCenters of America,3.428571,62.0,10404,26,14.0
AC,21,Associated Capital Group,2.333333,10.0,36,759,3.0
DNA,23,Ginkgo Bioworks,2.333333,10.0,403,175,3.0
LULU,26,lululemon athletica,2.000000,9.0,35,581,3.0
RNA,25,Avidity Biosciences,1.250000,9.0,-60,106,4.0
...,...,...,...,...,...,...,...
UP,17,Wheels Up,0.071429,13.0,70,46,14.0
DM,79,Desktop Metal,0.000000,3.0,7,79,3.0
MA,62,Mastercard,0.000000,4.0,100,133,4.0


In [ ]:
## Drop unnecessary data in preparation for plotting.

In [14]:
top_100_mentions_stocks_df.drop(columns=['rank','name','upvotes','rank_24h_ago','mentions','mentions_24h_ago'],inplace=True)
top_100_mentions_stocks_df

,mentions_volatility
ticker,
TA,3.428571
AC,2.333333
DNA,2.333333
LULU,2.000000
RNA,1.250000
...,...
UP,0.071429
DM,0.000000
MA,0.000000


In [ ]:
## Create variables based upon desired parameters.

In [15]:
top_10_mentions_stocks_df = top_100_mentions_stocks_df.head(10)
top_5_mentions_stocks_df = top_10_mentions_stocks_df.head(5)
less_mentioned_5_stocks_df = top_100_mentions_stocks_df.tail(5)

In [ ]:
## Plot Data 

In [16]:
# Returns the current local date
todays_date = date.today()

top_5_mentions_stocks_plot = top_5_mentions_stocks_df.hvplot(kind="bar", frame_width=380, xlabel='Ticker', ylabel='Social Mentions Volatility Score', title=f'Top Meme Stocks since {todays_date - timedelta(2)}')
top_5_mentions_stocks_plot

:Bars   [ticker]   (mentions_volatility)

In [ ]:
## Plot Data

In [17]:
less_mentioned_5_stocks_plot = less_mentioned_5_stocks_df.hvplot(kind="bar", frame_width=380, xlabel='Ticker', ylabel='Social Mentions Volatility Score', title=f'Less Social Mentions since {todays_date - timedelta(2)}')
less_mentioned_5_stocks_plot

:Bars   [ticker]   (mentions_volatility)

In [ ]:
## Convert dataframe datatype to string values for evaluation

In [18]:
top_10_mentions_stocks = top_10_mentions_stocks_df.index.astype('string')
top_10_mentions_stocks

Index(['TA', 'AC', 'DNA', 'LULU', 'RNA', 'DE', 'AI', 'GM', 'DOW', 'AAPL'], dtype='object', name='ticker')

In [ ]:
## Convert dataframe datatype to string values for evaluation

In [19]:
less_mentioned_5_stocks_df = less_mentioned_5_stocks_df.index.astype('string')
less_mentioned_5_stocks_df

Index(['UP', 'DM', 'MA', 'ES', 'ICE'], dtype='object', name='ticker')

In [ ]:
## Load dotenv environment to 'hide' folders containing private keys from the public after uploading to Github

In [20]:
load_dotenv('.env')

True

In [ ]:
## Call on keys in hidden folder to access the Alpaca API

In [21]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

In [ ]:
## GET used to call historic data from Alpaca. 'trade_count' used as a parameter and concatenated for display

In [22]:
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = 'v2')
 
three_days_back_date = todays_date - timedelta(days=7)

end_date = pd.Timestamp(todays_date, tz='America/New_York').isoformat()
start_date = pd.Timestamp(three_days_back_date, tz='America/New_York').isoformat() 

activity_top_1_stock_df = alpaca.get_bars(top_10_mentions_stocks[0:1], timeframe = '24Hour', start = start_date, end = end_date).df
activity_top_1_stock_df.drop(columns=['open','low','close','high','vwap','volume'],inplace=True)

activity_top_2_stock_df = alpaca.get_bars(top_10_mentions_stocks[1:2], timeframe = '24Hour', start = start_date, end = end_date).df
activity_top_2_stock_df.drop(columns=['open','low','close','high','vwap','volume'],inplace=True)

activity_top_3_stock_df = alpaca.get_bars(top_10_mentions_stocks[2:3], timeframe = '24Hour', start = start_date, end = end_date).df
activity_top_3_stock_df.drop(columns=['open','low','close','high','vwap','volume'],inplace=True)

activity_top_4_stock_df = alpaca.get_bars(top_10_mentions_stocks[3:4], timeframe = '24Hour', start = start_date, end = end_date).df
activity_top_4_stock_df.drop(columns=['open','low','close','high','vwap','volume'],inplace=True)

activity_top_5_stock_df = alpaca.get_bars(top_10_mentions_stocks[4:5], timeframe = '24Hour', start = start_date, end = end_date).df
activity_top_5_stock_df.drop(columns=['open','low','close','high','vwap','volume'],inplace=True)

top_5_stocks_activity_volatility_df = pd.concat([activity_top_1_stock_df['trade_count'], 
                                    activity_top_2_stock_df['trade_count'], 
                                    activity_top_3_stock_df['trade_count'],
                                    activity_top_4_stock_df['trade_count'],
                                    activity_top_5_stock_df['trade_count']], axis = 1, keys=top_10_mentions_stocks[0:5])

top_5_stocks_activity_volatility_df


ticker,TA,AC,DNA,LULU,RNA
timestamp,,,,,
2022-05-16 00:00:00+00:00,1145,121,61163,20955,4463
2022-05-17 00:00:00+00:00,2281,153,68748,28012,3935
2022-05-18 00:00:00+00:00,3211,169,53187,59312,4744
2022-05-19 00:00:00+00:00,2258,87,58629,50341,4368
2022-05-20 00:00:00+00:00,2572,83,53215,39772,2827


In [ ]:
## GET used to call historic data from Alpaca. 'high' used as a parameter and concatenated for display

In [23]:
prices_top_1_stock_df = alpaca.get_bars(top_10_mentions_stocks[0:1], timeframe = '24Hour', start = start_date, end = end_date).df
prices_top_1_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_top_2_stock_df = alpaca.get_bars(top_10_mentions_stocks[1:2], timeframe = '24Hour', start = start_date, end = end_date).df
prices_top_2_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_top_3_stock_df = alpaca.get_bars(top_10_mentions_stocks[2:3], timeframe = '24Hour', start = start_date, end = end_date).df
prices_top_3_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_top_4_stock_df = alpaca.get_bars(top_10_mentions_stocks[3:4], timeframe = '24Hour', start = start_date, end = end_date).df
prices_top_4_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_top_5_stock_df = alpaca.get_bars(top_10_mentions_stocks[4:5], timeframe = '24Hour', start = start_date, end = end_date).df
prices_top_5_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

top_5_stocks_volatility_df = pd.concat([prices_top_1_stock_df['high'], 
                                    prices_top_2_stock_df['high'], 
                                    prices_top_3_stock_df['high'],
                                    prices_top_4_stock_df['high'],
                                    prices_top_5_stock_df['high']], axis = 1, keys=top_10_mentions_stocks[0:5])

top_5_stocks_volatility_df

ticker,TA,AC,DNA,LULU,RNA
timestamp,,,,,
2022-05-16 00:00:00+00:00,37.195,40.01,3.18,308.660,14.06
2022-05-17 00:00:00+00:00,37.920,39.67,2.84,316.490,15.12
2022-05-18 00:00:00+00:00,36.290,38.76,3.00,300.060,14.59
2022-05-19 00:00:00+00:00,34.710,37.85,2.66,278.715,13.25
2022-05-20 00:00:00+00:00,33.930,38.24,2.71,280.860,13.20


In [ ]:
## GET used to call historic data from Alpaca. 'trade_count' used as a parameter and concatenated for display

In [24]:
prices_low_mentions_1_stock_df = alpaca.get_bars(less_mentioned_5_stocks_df[0:1], timeframe = '24Hour', start = start_date, end = end_date).df
prices_low_mentions_1_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_low_mentions_2_stock_df = alpaca.get_bars(less_mentioned_5_stocks_df[1:2], timeframe = '24Hour', start = start_date, end = end_date).df
prices_low_mentions_2_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_low_mentions_3_stock_df = alpaca.get_bars(less_mentioned_5_stocks_df[2:3], timeframe = '24Hour', start = start_date, end = end_date).df
prices_low_mentions_3_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_low_mentions_4_stock_df = alpaca.get_bars(less_mentioned_5_stocks_df[3:4], timeframe = '24Hour', start = start_date, end = end_date).df
prices_low_mentions_4_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

prices_low_mentions_5_stock_df = alpaca.get_bars(less_mentioned_5_stocks_df[4:5], timeframe = '24Hour', start = start_date, end = end_date).df
prices_low_mentions_5_stock_df.drop(columns=['open','low','close','trade_count','vwap','volume'],inplace=True)

top_5_stocks_low_mentions_volatility_df = pd.concat([prices_low_mentions_1_stock_df['high'], 
                                    prices_low_mentions_2_stock_df['high'], 
                                    prices_low_mentions_3_stock_df['high'],
                                    prices_low_mentions_4_stock_df['high'],
                                    prices_low_mentions_5_stock_df['high']], axis = 1, keys=less_mentioned_5_stocks_df[0:])

top_5_stocks_low_mentions_volatility_df

ticker,UP,DM,MA,ES,ICE
timestamp,,,,,
2022-05-16 00:00:00+00:00,2.58,2.11,333.550,90.120,98.28
2022-05-17 00:00:00+00:00,2.73,2.10,341.180,90.130,99.50
2022-05-18 00:00:00+00:00,2.68,2.17,341.595,90.045,98.89
2022-05-19 00:00:00+00:00,2.57,2.15,336.210,90.160,96.92
2022-05-20 00:00:00+00:00,2.55,2.16,339.610,90.440,97.89


In [ ]:
## Plot trade activity/top mentions stocks over 4 days

In [25]:
top_stocks_activity_volatility_df_plot = (1 + top_5_stocks_activity_volatility_df.pct_change()).cumprod().hvplot(frame_width=600,xlabel='Date',ylabel='Trade Activity Score', title='Volatility in Trade Activity  on Top Mentioned Stocks')
top_stocks_activity_volatility_df_plot

:NdOverlay   [ticker]
   :Curve   [timestamp]   (value)

In [ ]:
## Plot price volatility/top mentions stocks over 4 days. Visulaize using percent change

In [26]:
top_stocks_volatility_df_plot = (1 + top_5_stocks_volatility_df.pct_change()).cumprod().hvplot(frame_width=600,xlabel='Date',ylabel='High Price', title='Price Volatility in Top Mentioned Stocks')
top_stocks_volatility_df_plot

:NdOverlay   [ticker]
   :Curve   [timestamp]   (value)

In [ ]:
## Plot price volatility/top mentions stocks over 4 days. Visulaize 

In [27]:
top_stocks_volatility_df_plot = top_5_stocks_volatility_df.hvplot(frame_width=600,xlabel='Date',ylabel='High Price', title='Price Volatility in Top Mentioned Stocks')
top_stocks_volatility_df_plot

:NdOverlay   [ticker]
   :Curve   [timestamp]   (value)

In [ ]:
## Plot price volatility/less mentioned stocks over 4 days. Visulaize

In [28]:
top_5_stocks_low_mentions_volatility_plot = top_5_stocks_low_mentions_volatility_df.hvplot(frame_width=600,xlabel='Date',ylabel='High Price', title='Price Volatility in Less Mentioned Stocks')
top_5_stocks_low_mentions_volatility_plot

:NdOverlay   [ticker]
   :Curve   [timestamp]   (value)

In [ ]:
## Compare/contrast volatility vs mentions data in top stocks

In [29]:
summary_plot = top_stocks_activity_volatility_df_plot + top_5_mentions_stocks_plot
summary_plot

:Layout
   .NdOverlay.I :NdOverlay   [ticker]
      :Curve   [timestamp]   (value)
   .Bars.I      :Bars   [ticker]   (mentions_volatility)

In [30]:
top_stocks_volatility_df_plot 

:NdOverlay   [ticker]
   :Curve   [timestamp]   (value)

In [ ]:
## Compare/contrast volatility vs mentions data in less mentioned stocks

In [31]:
low_mentions_plot = top_5_stocks_low_mentions_volatility_plot + less_mentioned_5_stocks_plot
low_mentions_plot

:Layout
   .NdOverlay.I :NdOverlay   [ticker]
      :Curve   [timestamp]   (value)
   .Bars.I      :Bars   [ticker]   (mentions_volatility)